In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import pandas as pd
import pandasql as pds
import re
import numpy as np
import os
import matplotlib
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
path_chrome = "D:/Programming/Python/chromedriver.exe"
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

In [139]:
# url = "https://stackoverflow.com/questions/tagged/python"
# driver = webdriver.Chrome(path_chrome)
# driver.get(url)

In [144]:
# list of programming languages
driver = webdriver.Chrome(path_chrome)
driver.get("https://en.wikipedia.org/wiki/List_of_programming_languages")
languages = driver.find_elements(By.CSS_SELECTOR, 'div.div-col') # this makes a <list> of the <lists> of <programming languages> by <alphabet>
languages

In [158]:
langs = {}  # dictionary to store every language for each letter. this will be used to search them later in the tags.

"A.NET (A#/A sharp)\nA-0 System\nA+ (A plus)\nA++\nABAP\nABC\nABC ALGOL\nACC\nAccent (Rational Synergy)\nAce DASL (Distributed Application Specification Language)\nAction!\nActionScript\nActor\nAda\nAdenine (Haystack)\nAdvPL\nAgda\nAgilent VEE (Keysight VEE)\nAgora\nAIMMS\nAldor\nAlef\nALF\nALGOL 58\nALGOL 60\nALGOL 68\nALGOL W\nAlice (Alice ML)\nAlma-0\nAmbientTalk\nAmiga E\nAMOS (AMOS BASIC)\nAMPL\nAnalitik\nAngelScript\nApache Pig latin\nApex (Salesforce.com, Inc)\nAPL\nApp Inventor for Android's visual block language (MIT App Inventor)\nAppleScript\nAPT\nArc\nARexx\nArgus\nAssembly language (ASM)\nAutoHotkey\nAutoIt\nAutoLISP / Visual LISP\nAverest\nAWK\nAxum"

In [160]:
for l in languages:
    langs_letter = l.text.split("\n")  # splitting since the elements were lang1\nlang2\nlang3... in one large string rather than a string for every language. 
    langs[langs_letter[0][0]] = langs_letter  # dictionary[letter] = list_of_languages_in_that_letter
# langs = dictionary of languages per letter. ez clap. 

In [200]:
#k:[values]                                        stage 1: creating the .csv file with all the languages.
#pd.DataFrame(d.items())
langsdf = pd.DataFrame(columns = langs.keys())
langsdf
for ch in langsdf.columns:
    langsdf[ch] = pd.Series(langs[ch])  # list of languages in ch. need to somehow put it in the column in the df. 
langsdf.fillna("null", inplace = True)
langsdf.to_csv("D:/Programming/GitHub/Projects/data/pLanguages.csv")

In [192]:
#codes = driver.find_elements(By.CSS_SELECTOR, 'code')
driver.quit()

In [201]:
# getting the questions from searching the language in stack overflow. this is used to later search those same questions, click on them, and retrieve the code lines. 
pLanguages = pd.read_csv("D:/Programming/GitHub/Projects/data/pLanguages.csv")

In [2]:
pLanguages

NameError: name 'pLanguages' is not defined

In [32]:
# retrieves a dictionary of {P# Q# : code lines}. the dictionary: questions_and_codes.




lang = 'python'  # the label of the training data
path_chrome = "D:/Programming/Python/chromedriver.exe"
url = "https://stackoverflow.com/questions/tagged/{}?tab=frequent&page={}&pagesize=15".format(lang, 1)
url2 = "https://stackoverflow.com/questions/tagged/{}?tab=votes&page={}&pagesize=15".format(lang, 1)
driver = webdriver.Chrome(path_chrome)
questions_and_codes_new = {}  # empty dictionary. formatted as such: {"question":"list of shit ton of code lines"} this is the training data. 
pages_to_search = 25
driver.get(url)  # initialize driver onto the url. in this case, page 1 of the search pages.

In [32]:
for page in range(pages_to_search):
    Qindex = len(driver.find_elements(By.CSS_SELECTOR, 'h3.s-post-summary--content-title'))  # amount of questions in the page. 
    time.sleep(0.5)
    for i in range(Qindex):
        time.sleep(0.5)

        Qs = driver.find_elements(By.CSS_SELECTOR, 'h3.s-post-summary--content-title')  # list of questions in the page.
        thelink = Qs[i].find_element(By.CSS_SELECTOR, 'a.s-link').get_attribute("href")  # their hrefs
        driver.execute_script("window.open('');")  # opens a new tab
        driver.switch_to.window(driver.window_handles[1])  # switches to new tab
        driver.get(thelink)  # loading the question's href on it
        codes = driver.find_elements(By.CSS_SELECTOR, 'code')  # collecting code lines.
        # if "p{}-Q{}".format(page, i) in questions_and_codes.keys():
        #     continue
        try:
            questions_and_codes["p{}-Q{}".format(page, i)] = [code.text for code in codes]
        except:
            questions_and_codes["p{}-Q{}".format(page, i)] = ""
        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # switch to first tab. otherwise Qs still looks for the already-closed tabs and it errors
    time.sleep(0.5)
    driver.find_element(By.LINK_TEXT, 'Next').click()
questions_and_codes  # the dictionary of all questions and their code lines.    

C:\Users\Itamar\AppData\Local\Temp\ipykernel_9436\2631748831.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path_chrome)


In [42]:
# seek to improve this. i hate looking at triple "for". maybe it's okay but it doesnt look right so try and replace this.


codelines = []
for key in questions_and_codes.keys():
    codelist = questions_and_codes[key]
    for line in codelist:
        for inst in line.split("\n"):
            codelines.append(inst)
            
pd.DataFrame(pd.Series(codelines), columns = {'python'}).to_csv("D:/Programming/GitHub/projects/data/pythonCode.csv")



In [43]:
lang_df = pd.DataFrame(pd.Series(codelines), columns = {'python'})
lang_df

,python
0,pandas
1,pandas
2,import pandas as pd
3,"df = pd.DataFrame({'user': ['Bob', 'Jane', 'Al..."
4,"'income': [40000, 50000, 42..."
...,...
109764,"with open(file_name, ""rU"") as read_file:"
109765,"for i, row in enumerate(read_file, 1):"
109766,#do something
109767,#i in line of that line


In [13]:
pydf = pd.read_csv("d:/programming/github/projects/data/PythonCode.csv")
jdf = pd.read_csv("d:/programming/github/projects/data/JavaCode.csv")
cdf = pd.read_csv("d:/programming/github/projects/data/CCode.csv")
unnamed = pydf.columns[0]
for df in [pydf, jdf, cdf]:
    del df[unnamed]
    df['language'] = df.columns[0]
    df.rename(columns = {df.columns[0]:'line'}, inplace = True)
pydf

,line,language
0,pandas,python
1,pandas,python
2,import pandas as pd,python
3,"df = pd.DataFrame({'user': ['Bob', 'Jane', 'Al...",python
4,"'income': [40000, 50000, 42...",python
...,...,...
109764,"with open(file_name, ""rU"") as read_file:",python
109765,"for i, row in enumerate(read_file, 1):",python
109766,#do something,python
109767,#i in line of that line,python


In [16]:
df = pd.concat([pydf, jdf, cdf]).reset_index()
del df['index']
df

,line,language
0,pandas,python
1,pandas,python
2,import pandas as pd,python
3,"df = pd.DataFrame({'user': ['Bob', 'Jane', 'Al...",python
4,"'income': [40000, 50000, 42...",python
...,...,...
252226,2,C
252227,0,C
252228,1,C
252229,0,C


In [1]:
# the data prediction:
# step 1: separate the words.
# step 2: take out tokens that don't add new insight to the label.




# how to do: 
# make a histogram with the words? 
# then take out keys that would not be relevant, or that would have the same amount of entries in every language?
# then maybe find a way to match regex patterns between the way loops are built in each language and differentiate by that? 
# something along the lines of [\w+]\ \=\ \{(.*\:.*)+\} to match dictionary pattern (char = {(anything:anything)any amount of times larger than 1}) etc...

In [142]:
# consider just runnning over all questions in stack overflow and based on their tags put them in a dictionary for their language. that could save some time with doing this process for every language, 
# but it could also be more costly as it would have to classify based on a shit ton of tags. sometimes "error-handling, shutil, jinja2", all useless bullshit that aren't really languages. 
# plan your database. 

In [ ]:
# how about one that clicks each question and collects the code?
# search quetsions by language from stackoverflow.

In [9]:
lang = 'python'  # the label of the training data
path_chrome = "D:/Programming/Python/chromedriver.exe"
url = "https://stackoverflow.com/questions/tagged/{}?tab=newest&page={}&pagesize=15".format(lang, 1)
driver = webdriver.Chrome(path_chrome)
driver.get(url)  # initialize driver onto the url. in this case, page 1 of the search pages.
questions_and_codes = {}  # empty dictionary. formatted as such: {"question":"list of shit ton of code lines"} this is the training data. 
pages_to_search = 1
for i in range(pages_to_search):  # the range indicates the amount of pages to look through. 
    page_qs = driver.find_elements(By.CSS_SELECTOR, 'h3.s-post-summary--content-title')  # the questions in current page. 
    for Q in driver.find_elements(By.CSS_SELECTOR, 'h3.s-post-summary--content-title'):
        time.sleep(0.5)
        Q.click()
        time.sleep(0.5)
        try:
            questions_and_codes[Q.text] = driver.find_elements(By.CSS_SELECTOR, 'code')
        except:
            driver.execute_script("window.history.go(-1)")  # previous page  
            time.sleep(0.6)
            continue
        time.sleep(0.5)
        driver.execute_script("window.history.go(-1)")  # previous page
        time.sleep(0.5)
    driver.find_element(By.LINK_TEXT, 'Next').click()
        

C:\Users\Itamar\AppData\Local\Temp\ipykernel_9436\2410237184.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path_chrome)


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x010278B3+2193587]
	Ordinal0 [0x00FC0681+1771137]
	Ordinal0 [0x00ED41A8+803240]
	Ordinal0 [0x00ED6BB4+814004]
	Ordinal0 [0x00ED6A72+813682]
	Ordinal0 [0x00ED6D00+814336]
	Ordinal0 [0x00F03F19+999193]
	Ordinal0 [0x00EF9146+954694]
	Ordinal0 [0x00F1D7AC+1103788]
	Ordinal0 [0x00EF8C04+953348]
	Ordinal0 [0x00F1D9C4+1104324]
	Ordinal0 [0x00F2DAE2+1170146]
	Ordinal0 [0x00F1D5C6+1103302]
	Ordinal0 [0x00EF77E0+948192]
	Ordinal0 [0x00EF86E6+952038]
	GetHandleVerifier [0x012D0CB2+2738370]
	GetHandleVerifier [0x012C21B8+2678216]
	GetHandleVerifier [0x010B17AA+512954]
	GetHandleVerifier [0x010B0856+509030]
	Ordinal0 [0x00FC743B+1799227]
	Ordinal0 [0x00FCBB68+1817448]
	Ordinal0 [0x00FCBC55+1817685]
	Ordinal0 [0x00FD5230+1856048]
	BaseThreadInitThunk [0x7577FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76EF7A9E+286]

In [8]:
questions_and_codes

{}

In [255]:
driver.find_element(By.XPATH, '//*[@id="question-summary-73601146"]/div[2]/h3/a').click()

In [258]:
for i in driver.find_elements(By.CSS_SELECTOR, 'h3.s-post-summary--content-title'):
    print(i.text)
    

In [261]:
driver.execute_script("window.history.go(-1)")  # previous page

In [ ]:
questions = []  # want to go through all the pages to get the questions. then i want to click them one by one in the next kernel and get codes. 

page_length = int(driver.find_element(By.XPATH, '//*[@id="mainbar"]/div[7]/a[5]').text)
for i in range(10):
    try:
        qs = driver.find_elements(By.CSS_SELECTOR, 'a.s-link')
    except:
        break
        print("no data sir")
    for q in qs:
        if q.text != '' and q.text not in questions:
            questions.append(q.text)
    time.sleep(0.6)
    driver.find_element(By.LINK_TEXT, 'Next').click()
    time.sleep(0.5)
questions